# Data Insights Analysis using Crew AI

In [1]:
!pip install crewai langchain_community crewai_tools

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import yaml
from crewai import Agent, Task, Crew

In [5]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')

# Loading Agents and Tasks YAML files

In [6]:
# Define file paths for YAML configurations
files = {
    'agents': './config/agents.yaml',
    'tasks': './config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Using  FileReadTool

In [7]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

# Creating Agents ,Tasks and Crew

In [10]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=False
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


# Testing our Crew

In [11]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
T0001,C0511,API Issue,I'm pleased with how my issue was handled. Thanks!,High,2023-03-25,24

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  9.0  │    9.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  9.0  │    9.0     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.50  │    8.5     │                     │  │  ┃
┃ Execution Time (s) │  110  │    110     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

# Training your Crew and Agents

In [12]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
T0001,C0511,API Issue,I'm pleased with how my issue was handled. Thanks!,High,2023-03-25,24

# Compairing New test Results

In [15]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
T0001,C0511,API Issue,I'm pleased with how my issue was handled. Thanks!,High,2023-03-25,24

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  9.0  │    9.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  9.0  │    9.0     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.56  │    8.6     │                     │  │  ┃
┃ Execution Time (s) │  279  │    279     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

# kicking off Crew

In [16]:
result = support_report_crew.kickoff()

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Thought: I need to gather the necessary data related to the support tickets to provide actionable suggestions.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comm

# Result

In [17]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

**Final Report on Support System Performance**

---

### Issue Classification Results
| Issue Type         | Frequency | High Priority | Medium Priority | Low Priority | Critical Priority |
|---------------------|-----------|---------------|-----------------|--------------|-------------------|
| API Issues          | 9         | 5             | 2               | 1            | 1                 |
| Login Issues        | 7         | 3             | 0               | 3            | 1                 |
| Report Generation    | 6         | 2             | 2               | 2            | 0                 |
| Data Import         | 10        | 4             | 4               | 2            | 0                 |
| Feature Requests     | 7         | 3             | 3               | 1            | 1                 |
| Billing Issues      | 10        | 4             | 3               | 3            | 2                 |
| UI Bugs             | 5         | 2             | 2               | 1            | 1                 |

![Issue Distribution Chart](Issue_Distribution_Chart.png)

![Priority Levels Chart](Priority_Levels_Chart.png)

---

### Agent Performance
| Agent ID | Average Resolution Time (minutes) | Average Satisfaction Rating | Number of Resolved Tickets | Number of Tickets Assigned |
|----------|-------------------------------------|----------------------------|---------------------------|---------------------------|
| A001     | 832.67                              | 2.33                       | 12                        | 16                        |
| A002     | 745.25                              | 2.25                       | 9                         | 12                        |
| A003     | 687.80                              | 2.80                       | 10                        | 12                        |
| A004     | 716.35                              | 3.11                       | 16                        | 20                        |
| A005     | 799.14                              | 3.00                       | 7                         | 8                         |

![Agent Performance Chart](Agent_Performance_Chart.png)

---

### Customer Satisfaction Over Time
| Date       | Average Satisfaction Rating | Total Number of Responses |
|------------|----------------------------|---------------------------|
| 2023-01-01 | 3.25                       | 8                         |
| 2023-02-01 | 2.80                       | 10                        |
| 2023-03-01 | 2.90                       | 12                        |
| 2023-04-01 | 2.67                       | 10                        |
| 2023-05-01 | 2.90                       | 12                        |
| 2023-06-01 | 2.70                       | 10                        |
| 2023-07-01 | 3.00                       | 10                        |

![Customer Satisfaction Ratings Over Time](Customer_Satisfaction_Over_Time_Chart.png)

---

### Suggested Actions
1. **API Issues**
   - Schedule a follow-up call for customer satisfaction.
   - Evaluate the API workflow for delays and enhance agent training.
   - Implement proactive monitoring for recurrent API issues.

2. **Login Issues**
   - Investigate error logs and provide a comprehensive FAQ.
   - Automate initial response and ticket creation for faster resolution.
   - Continue timely interventions as proven successful.

3. **Report Generation**
   - Offer additional training on report generation tools.
   - Establish stricter SLAs for report queries.
   - Implement active monitoring for common failures.

4. **Data Import**
   - Document as a best practice model.
   - Establish rapid response teams for data imports.
   - Conduct a root cause analysis for common errors.

5. **Feature Requests**
   - Follow up with the customer for updates.
   - Create a special evaluation process for critical requests.
   - Develop a roadmap addressing frequent requests.

6. **Billing Issues**
   - Provide frequent updates on billing queries.
   - Enhance agent training on common billing issues.
   - Implement a feedback loop post-billing resolution.

7. **UI Bugs**
   - Address urgent bugs with a dedicated team.
   - Suggest a thorough UI review for persistent complaints.
   - Create a bug report dashboard for better tracking.

8. **General Recommendations**
   - Analyze ticket responses and agent performance metrics.
   - Introduce customer feedback loops post-resolution.
   - Utilize insights from resolved tickets for agent training and documentation.

---

This report aims to provide valuable insights into the performance of the support system, including issue classification, agent performance, customer satisfaction trends, and actionable suggestions for improvement.